# 02 Generate and tag sentences

#### Connection to Google Drive - Only execute if you are running this on GD

In [ ]:
# set up connection to your google drive
# please click on the link generated and enter the authorisation code

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Colab/statbot

/content/drive/MyDrive/Colab/statbot


In [ ]:
! pip install wget


## Script A02

This script generates the training dataset, containing questions and the   corresponding tags (such as granularity and datasets) in the Spacy-format so it can be used for the training.

It loads a csv-file with pre-written questions that contain brackets to insert elements such as {dataset} or {year} or {granularity} etc.  `(see input/question_generator.csv)` Currently, we only have seven questions, please feel free to add more. For every dataset we have, several questions are being generated randomly.

A lot could be improved in this script. At the moment, it only takes data of the so called "Gemeindeportraet" of the Statistics Office of the Canton of Zurich. This is because those data variables are in the same format and use the same variable names. We want of course to get away from this structure and use all kind of statistical OGD.

## Thoughts for improvement

- Include other useful **custom tags**
- **Insert important elements** correctly, such as year data, locality level, etc.
- Improve **generating the questions**: i.e. by insterting many more ways of saying 'take the three first elements', 'take all elements larger than X', etc. etc.
- Optional: Try and **tag every single** word in the datasets as dataset element to improve recognition (and then later use this to attribute them to the right dataset) <br><br>

- Outlook for later: Try and **combine this script with the generation of training data** so it can be used for an approach with seq2SQL or similar


In [ ]:
from __future__ import print_function, unicode_literals
import pandas as pd
import numpy as np
from random import sample
import io, csv
import re
import random
import json

In [ ]:
def uniform_cleaning(str_in):
    str_in=str_in.replace("-","")
    str_in=str_in.replace("(","")
    str_in=str_in.replace(")","")
    str_in=str_in.replace("ü","ue")
    str_in=str_in.replace("ä","ae")
    str_in=str_in.replace("ö","oe")
    return(str_in)
    
    

In [ ]:
random.seed(1)

df=pd.read_csv("data/datasets_overview.csv")

#one more prepatation
df['question_type'] = df['dataset_title'].str.extract(r"\[(.*?)\]", expand=False)
df['question_type'] = np.where(df['question_type']== '%', "percent", "cardinal")
df['dataset_title'] = df['dataset_title'].str.replace(r"\[(.*?)\]", "")
#structurize relevant informations such as variables

#load content of datasets


# load template sentences 
sentences=pd.read_csv("input/question_generator.csv")
out_sentences=[]

for i in df['index'].unique():
    # for each dataset of the gemeindeportraet 
    filename="data/"+str(i)+".csv" 
    with io.open(filename, 'r', encoding="latin-1") as csvfile:
        dialect = csv.Sniffer().sniff(csvfile.readline(), [',',';'])
        csvfile.seek(0)
        mydelimiter=dialect.delimiter
    try:
      # open dataset 
        this_data=pd.read_csv(filename,delimiter=dialect.delimiter)
        vars=df['var'].loc[df['index'] == i]
        dataset_title=df['dataset_title'].loc[df['index']==i].any()
        question_type=df['question_type'].loc[df['index']==i].any()
        # at the moment: only take columns containing the main value of the gemeindeportraet dataset----------
        if vars.str.contains("INDIKATOR_VALUE").sum():
            #title of the column 
            main=uniform_cleaning(dataset_title.strip())
            # values of the column  
            filter_vars=vars
            #print(filter_vars)
        else:
          #skip this column (temporary so all variables look alike) 
            continue 
            print("ATTRIBUTING MAIN VARIABLE TO: ",dataset_title)
            length_title_string=len(dataset_title.split())
            highest_similarity=0
            which=None
            for var in vars:
                temp_string=dataset_title+" "+var
                #print(temp_string)
                doc = nlp(temp_string)
                assert len(doc.vector) == len(doc[0].vector)
                calc_similarity=doc[:length_title_string].similarity(doc[length_title_string:])
                if  calc_similarity> highest_similarity:
                    highest_similarity=calc_similarity
                    which=var
            print("Highest similarity:",var,highest_similarity)
            main=uniform_cleaning(var)
            vars=vars.tolist()
            print(vars)
            print(main)
            filter_vars=vars.remove(main)

        #the following temporary because it is standardized
        filter_vars=list(filter_vars)
        filter_vars.append("")
        try:
            filter_vars.remove("INDIKATOR_JAHR")
            filter_vars.remove("GEBIET_NAME")
            filter_vars.remove("BFS_NR")
            filter_vars.remove("INDIKATOR_VALUE")
        except:
            print("variable removing empty")

        # generate sentences 
        for sentence in sentences['question'].loc[sentences['main_type'] == question_type]:
            out_entities = []
            # replace placeholder with column title extracted above  
            sentence=sentence.replace("{main}",main)
            sentence=sentence.replace("{localitylevel}","")#at the moment empty
            #TODO either one locality, one level, or several localities
            random_value=sample(["one locality","one level","several localities"],1)[0]
            if random_value=="one locality":
                locality_insert="in "+sample(list(this_data['GEBIET_NAME']),1)[0]
            if random_value=="one level":
                locality_insert=sample(["für den gesamten Kanton","im Kanton Zürich","auf Bezirksebene",
                "für alle Bezirke","pro Bezirk","auf Gemeindeebene","für alle Gemeinden","pro Gemeinde"],1)[0]
            if random_value=="several localities":
                locality_insert=""
                local_loop=sample([1,2,3],1)[0]
                for local in range(0,local_loop):
                    if local!=0 and local!=(local_loop-1):
                        locality_insert+=", "
                    if local!=0 and local==(local_loop-1):
                        locality_insert+=" und "
                    locality_insert+=sample(list(this_data['GEBIET_NAME']),1)[0]
            sentence=sentence.replace("{locality}",locality_insert)
            sentence=sentence.replace("{yeartime}","")#TODO no,aktuellste,neuste, value from list, from to year
            sentence=sentence.replace("{filter}",sample(list(filter_vars),1)[0])

            for mat in re.findall(r'.*?\[(.*)].*', sentence):
                which_part=sample([1,2],1)

                if which_part==1:
                    sentence=sentence.replace("["+mat+"]",mat.partition("|")[0])
                    #sentence=re.sub("["+mat+"]",mat.partition("|")[0],sentence)
                else:
                    sentence=sentence.replace("["+mat+"]",mat.partition("|")[2])
                    #sentence=re.sub("["+mat+"]",mat.partition("|")[2],sentence)


            #now the symbol - has to be deleted as it gives issues
            sentence=uniform_cleaning(sentence)


            #### TAGGING

            #1) GRAN
            # check if sentence contains granularity 
            match_span = re.search(r'\bGEMEINDE\b|\bGEMEINDEN\b|\bGEMEINDEEBENE\b|\bBEZIRK\b|\bBEZIRKSEBENE\b|\bBEZIRKE\b|\bKANTON\b|\bKANTONSEBENE\b|\bREGION\b', sentence,flags=re.IGNORECASE)
            if match_span is not None:
                match_span=match_span.span()
                out_entities.append((match_span[0], match_span[1], "GRAN"))

            #2) DATASET
            print(main)
            match_span = re.search(main,sentence,flags=re.IGNORECASE) 
            print(match_span)
            if match_span is not None:
                match_span=match_span.span()
                out_entities.append((match_span[0], match_span[1], "DATA"))


            out_sentences.append((sentence, {"entities": out_entities}))

    # for colab: list files that cannot be opened by google colab
    except:
        errors.append(i)
            

In [ ]:
# save the generated sentences 
print(out_sentences[:10])
with open("input/tagged_sentences.json","w",encoding='utf-8') as outfile:
    json.dump(out_sentences, outfile, ensure_ascii=False)


In [ ]:
# look at some sentences
print(out_sentences[1])
print(out_sentences[1][0][10])
print(out_sentences[1][0][12-1])